# eSports Earnings - Data Analysis

## Import necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

### Format options

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
plt.style.use('seaborn')

In [4]:
#plt.style.available

# 

## Load the Data

In [5]:
# Dataset: https://www.kaggle.com/jackdaoud/esports-earnings-for-players-teams-by-game

In [6]:
path = './Datasets/eSports/'

In [7]:
countries = pd.read_csv(path + 'country-and-continent-codes-list.csv')
players = pd.read_csv(path + 'highest_earning_players.csv')
teams = pd.read_csv(path + 'highest_earning_teams.csv')

FileNotFoundError: [Errno 2] No such file or directory: './Datasets/eSports/country-and-continent-codes-list.csv'

# 

## Data Exploratory

In [ ]:
countries.head()

In [ ]:
countries['Country_Number'].isnull().sum()

In [ ]:
countries[countries['Country_Number'].isnull()]

In [ ]:
countries.shape

In [ ]:
players.head()

In [ ]:
players.shape

In [ ]:
teams.head()

In [ ]:
teams.shape

# 

In [ ]:
countries.info(memory_usage='deep')

In [ ]:
countries.memory_usage(deep=True)

#

In [ ]:
players.info(memory_usage='deep')

In [ ]:
players.memory_usage(deep=True)

#

In [ ]:
teams.info(memory_usage='deep')

In [ ]:
teams.memory_usage(deep=True)

In [ ]:
# we could change some types e.g.: game and genre as category but here are only 400 KB

# 

In [ ]:
countries.isnull().sum()

In [ ]:
print('Missing Values (in countries DataFrame)', end='\n\n')
for col in countries.columns:
    miss_val = countries[col].isnull().mean()
    if len(str(miss_val * 100).split('.')[0]) == 1:
        print(f'[ {round(miss_val * 100, 1)}%]', col)
    else:
        print(f'[{round(miss_val * 100, 1)}%]', col)
    

In [ ]:
countries.duplicated().sum()

In [ ]:
players.isnull().sum()

In [ ]:
players.duplicated().sum()

In [ ]:
teams.isnull().sum()

In [ ]:
teams.duplicated().sum()

## Data Cleaning

In [ ]:
players.head()

In [ ]:
players.groupby(['Genre', 'Game'])['PlayerId'].count()

In [ ]:
print('So.. we have 100 players per game!')

# 

## Data Transformation

In [ ]:
players.head()

In [ ]:
players['CountryCode'].str.upper()

In [ ]:
players['CountryCode'] = players['CountryCode'].str.upper()

# 

In [ ]:
countries.head()

In [ ]:
filtered = countries['Two_Letter_Country_Code'].notnull()     # if we don't use that filter, we will get an error about NaN values
countries[filtered]['Two_Letter_Country_Code'].apply(lambda x: x.upper())

In [ ]:
countries['Two_Letter_Country_Code'] = countries[filtered]['Two_Letter_Country_Code'].apply(lambda x: x.upper())

# 

In [ ]:
merge_data = pd.merge(countries, players, left_on='Two_Letter_Country_Code', right_on='CountryCode')

In [ ]:
merge_data = merge_data.loc[:, ['PlayerId', 'NameFirst', 'NameLast', 'CurrentHandle', 'TotalUSDPrize', 'Game', 'Genre','Continent_Name', 'Country_Name', 'Country_Number']]

In [ ]:
merge_data.head()

# 

## Visualization

### Top Games Graph

In [ ]:
top_games = players.groupby('Game')['TotalUSDPrize'].sum().sort_values(ascending=False)

In [ ]:
top_games

In [ ]:
fig, ax = plt.subplots(figsize=(8,5), dpi=100)

plt.bar(top_games.index, top_games)


y_position = 3 * (10 ** 6)
for idx, val in enumerate(top_games):
    ax.text(idx, val + y_position, str(round(val / 10 ** 6, 1)) + 'M', ha='center', color='#36518B', fontweight='bold', fontsize=9.5)


plt.title('Top Games by Earnings', fontsize=14, color='#802323', fontweight='bold')
plt.ylabel('Total Earnings (in milion)', fontsize=12.5, color='#802323')

x = 10 ** 8    # 100,000,000
plt.yticks([0.25*x, 0.5*x, 0.75*x, 1*x, 1.25*x, 1.5*x, 1.75*x, 2*x])
plt.xticks(rotation=90)

#plt.grid(axis='y')
plt.show()

# 

### Top 10 Players (by Earnings)

In [ ]:
players.head()

In [ ]:
top_10_players = players.sort_values('TotalUSDPrize', ascending=False).head(10)

In [ ]:
top_10_players = top_10_players.sort_values('TotalUSDPrize')

In [ ]:
top_10_players

In [ ]:
fig, ax = plt.subplots(figsize=(8,6), dpi=90)

plt.barh( '(' + top_10_players['Game'] + ') ' + top_10_players['CurrentHandle'], top_10_players['TotalUSDPrize'])


for idx, val in enumerate(top_10_players['TotalUSDPrize']):
    ax.text(val - 10 ** 5, idx - 0.1, str(round(val / 10 ** 6, 2)) + 'M', ha='right', fontsize=13, color='white',fontweight='bold')


plt.title('Top 10 Player by Earnings', color='#843030', fontsize=15, fontweight='bold')
plt.ylabel('(Game) Nick Name', color='#843030', fontsize=12, fontweight='bold')
plt.xlabel('Earnings (in millions)',  color='#843030', fontsize=12, fontweight='bold')

plt.show()

# 

In [ ]:
merge_data.head()

In [ ]:
earnings_by_country = merge_data.groupby('Continent_Name')['TotalUSDPrize'].sum()

In [ ]:
earnings_by_country

In [ ]:
fig, ax = plt.subplots(figsize=(5,4), dpi=100)
fig.subplots_adjust(0.3, 0, 1, 1)

plt.pie(earnings_by_country, labels=earnings_by_country.index, autopct='%.2f%%')

plt.title('Earnings by Country', color='#843030', fontsize=14, fontweight='bold')


plt.legend(loc='upper right',
           labels=['{}: ${} millions'.format(name, round(val / 10 ** 6, 2)) for name, val in zip(earnings_by_country.index, earnings_by_country)],
           bbox_to_anchor=(1.6,1),
           bbox_transform=fig.transFigure)

plt.show()

# 

### What is the proportion of earnings per genre for the top 10 games in terms of both teams and players?

In [ ]:
merge_data.head()

In [ ]:
teams.head()

In [ ]:
teams_groupedby_genre = teams.groupby('Genre')['TotalUSDPrize'].sum()

In [ ]:
teams_groupedby_genre

In [ ]:
players_groupedby_genre = players.groupby('Genre')['TotalUSDPrize'].sum()

In [ ]:
players_groupedby_genre

In [ ]:
total_earnings_by_genre = teams_groupedby_genre + players_groupedby_genre

In [ ]:
total_earnings_by_genre

In [ ]:
for idx, row in enumerate(total_earnings_by_genre):
    percent = str(round(row / total_earnings_by_genre.sum() * 100, 2)) + '%'
    
    if len(percent.split('.')[0]) == 2:
        print('[{1}] {0}'.format(total_earnings_by_genre.index[idx], percent))
    else:
        print('[ {1}] {0}'.format(total_earnings_by_genre.index[idx], percent))

# 

### Which game is most popular in terms of number of tournaments?

In [ ]:
teams.head()

In [ ]:
most_popular_game = teams.groupby('Game')['TotalTournaments'].sum().sort_values(ascending=False)

In [ ]:
most_popular_game.head(1)

# 

### Best 3 Teams by Game

In [ ]:
teams.head()

In [ ]:
bestTeamsByGame = teams.loc[:, ['Game','TeamName','TotalUSDPrize']].sort_values(['Game','TotalUSDPrize'], ascending=[True,False]).groupby('Game').head(3)

In [ ]:
bestTeamsByGame

In [ ]:
bestTeamsByGame = bestTeamsByGame.sort_values(['Game','TotalUSDPrize'], ascending=[True, False])

In [ ]:
bestTeamsByGame

In [ ]:
merge_data

In [ ]:
filtered = merge_data.loc[:, ['CurrentHandle', 'Continent_Name','TotalUSDPrize']]
hightest_player_by_continent = filtered.sort_values(['Continent_Name', 'TotalUSDPrize'], ascending=[True, False]).groupby('Continent_Name').head(5)

In [ ]:
hightest_player_by_continent = hightest_player_by_continent.sort_values(['Continent_Name', 'TotalUSDPrize'], ascending=[False, True]).set_index('Continent_Name')

In [ ]:
hightest_player_by_continent